From the example at https://huggingface.co/facebook/mgenre-wiki

paper https://direct.mit.edu/tacl/article/doi/10.1162/tacl_a_00460/110051/Multilingual-Autoregressive-Entity-Linking


## install

get the genre code from https://github.com/facebookresearch/GENRE 
then do pip3 install . 

and do pip3 install marisa-trie


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/mgenre-wiki")

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mgenre-wiki").eval()

# from genre.trie import Trie, MarisaTrie
# import pickle 

#with open("mGENRE/titles_lang_all105_marisa_trie_with_redirect.pkl", "rb") as f:
#    trie = pickle.load(f)

/home/p107437/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
#sentences = ["[START] Einstein [END] era un fisico tedesco."]
# sentences = ["Ze was getrouwd met [START] Ron Graham [END] ."]
# sentences = ["De wedstrijd kende een finale rijk aan schermutselingen, tot uiteindelijk [START] Stijn Devolder [END], vorige week reeds 3e in de Ronde van Zwitserland en dus in bloedvorm, er op 9 km van de streep vandoor ging."]
sentences = ['Een confrontatie waar naar wordt uitgekeken is die tussen Agustín Delgado en Artur Boruc. [START] Boruc [END] had het in de beginfase een stuk drukker dan zijn collega Cristian Mora die voor de gelegenheid de Ecuadoraanse vlag op zijn wangen had laten aanbrengen.']
sentences = tokenizer(sentences, return_tensors="pt")

constrained beam search still does not work, trie gives an error...

In [3]:
outputs = model.generate(
    **sentences, 
    num_beams=5,
    num_return_sequences=5,
    # OPTIONAL: use constrained beam search
    # prefix_allowed_tokens_fn=lambda batch_id, sent: trie.get(sent.tolist()),
)

/home/p107437/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [4]:
results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(results)

['Artur Boruc >> nl', 'Boruc >> nl', 'Arthur Boruc >> nl', 'Artur Boruc >> en', 'Artur Boruc >> es']


In [1]:
import requests

S = requests.Session()

URL = "https://nl.wikipedia.org/w/api.php"

PARAMS = {
    "action": "query",
    "prop": "pageprops",
    "format": "json"
}

def find_qid(results) :
    qid = 0
    for result in results:
        if result.endswith('nl') and not(qid) : 
            label = result.split(' >> ')[0]
            PARAMS['titles'] = label
            Wikipedia_data = S.get(url=URL, params=PARAMS).json()
            Pages = Wikipedia_data['query']['pages']
            for (key,val) in Pages.items() : 
                try : 
                    qid = val['pageprops']['wikibase_item']
                    break
                except : 
                    True
    if qid == 0 :
        print(results)
    return(qid)

In [ ]:
find_qid(results)

In [2]:
import WINNL_Utils
    

In [3]:
WINNL_Utils.winnl2mgenre(4125)

[(" [START] Weyts [END] roept de slachthuissector op de ' rotte appels ' eruit te pikken :",
  'Q2271035')]

In [4]:
import Linking_eval_Utils as eval

def evaluate_winnl(Item) :
    Annotations = WINNL_Utils.winnl2mgenre(Item)
    for (String,QID) in Annotations :
        sentences = tokenizer([String], return_tensors="pt")
        outputs = model.generate(
            **sentences, 
            num_beams=5,
            num_return_sequences=5
        )
        results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        print(results)
        predicted = find_qid(results)
        print(predicted,QID)

def evaluate_winnl_items(First,Last) :
    score0 = {'overlap': 0, 'system': 0, 'gold': 0}
    for Id in list(range(First,Last)) :
        print(Id)
        gold = WINNL_Utils.winnl_annotations(Id)
        #print(gold)
        Annotations = WINNL_Utils.winnl2mgenre(Id)
        #print(Annotations)
        if gold and 'Q404' not in gold : 
            system = set()
            for (String,QID) in Annotations :
                sentences = tokenizer([String], return_tensors="pt")
                outputs = model.generate(
                    **sentences, 
                    num_beams=5,
                    num_return_sequences=5
                )
                results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
                predicted = find_qid(results)
                system.add(predicted)
            #print(system)
            eval.update_score(system,gold,score0)
            print(score0)
    eval.print_score(score0)
        

In [6]:
evaluate_winnl(4127)

/home/p107437/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Animal Rights >> nl', 'Animal Rights (Verenigde Staten) >> nl', 'Animal Rights (Nederland) >> nl', 'Animal Rights (Verenigd Koninkrijk) >> nl', 'Animal Rights (band) >> nl']
Q41444901 Q41444901
['Bite Back >> nl', 'Bite Back >> en', 'Bite back >> nl', 'Bite Back >> de', 'Bite Back >> fr']
Q21583395 Q21583395


In [7]:
evaluate_winnl_items(4124,4224)

4124
4125
['J.H. Weyts >> nl', 'Herman Weyts >> nl', 'J.J.H. Weyts >> nl', 'Judith Weyts >> nl', 'Franciscus Weyts >> nl']
{'overlap': 0, 'system': 1, 'gold': 1}
4126
{'overlap': 1, 'system': 2, 'gold': 2}
4127
{'overlap': 3, 'system': 4, 'gold': 4}
4128
{'overlap': 4, 'system': 5, 'gold': 5}
4129
{'overlap': 7, 'system': 8, 'gold': 8}
4130
{'overlap': 8, 'system': 9, 'gold': 9}
4131
{'overlap': 10, 'system': 11, 'gold': 11}
4132
{'overlap': 11, 'system': 12, 'gold': 12}
4133
{'overlap': 12, 'system': 13, 'gold': 13}
4134
4135
{'overlap': 13, 'system': 14, 'gold': 14}
4136
4137
{'overlap': 15, 'system': 16, 'gold': 16}
4138
{'overlap': 17, 'system': 18, 'gold': 18}
4139
{'overlap': 17, 'system': 19, 'gold': 19}
4140
{'overlap': 18, 'system': 20, 'gold': 20}
4141
4142
{'overlap': 19, 'system': 21, 'gold': 21}
4143
['Hit FM (Nederland) >> nl', 'Hit FM >> nl', 'Hit FM (België) >> nl', 'Hitfm >> nl', 'HitFM (Nederland) >> nl']
{'overlap': 19, 'system': 22, 'gold': 22}
4144
4145
4146
{'over

In [30]:
Linking_eval_Utils.init_score()

{'overlap': 0, 'system': 0, 'gold': 0}

In [33]:
score = Linking_eval_Utils.init_score()

In [34]:
score

{'overlap': 0, 'system': 0, 'gold': 0}